In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

MA_BASE = "/home/rpat638/econ470/a0/work/ma-data/ma"
ENROLL  = f"{MA_BASE}/enrollment/Extracted Data"
SA      = f"{MA_BASE}/service-area/Extracted Data"
PEN     = f"{MA_BASE}/penetration/Extracted Data"
STARS   = f"{MA_BASE}/star-ratings/Extracted Star Ratings"
BENCH   = f"{MA_BASE}/benchmarks"

OUT_DIR = "data/output"
os.makedirs(OUT_DIR, exist_ok=True)

YEARS = list(range(2010, 2016))
print("Setup complete.")
print(f"Output will be saved to: {os.path.abspath(OUT_DIR)}")

Setup complete.
Output will be saved to: /home/rpat638/econ470/a0/work/hwk3/data/output


In [ ]:
RATING_VARS = [
    "breastcancer_screen", "rectalcancer_screen", "flu_vaccine",
    "physical_monitor", "specialneeds_manage", "older_medication",
    "older_pain", "osteo_manage", "diabetes_eye", "diabetes_kidney",
    "diabetes_bloodsugar", "ra_manage", "falling", "bladder",
    "medication", "statin", "nodelays", "carequickly",
    "customer_service", "overallrating_care", "overallrating_plan",
    "coordination", "complaints_plan", "leave_plan", "improve",
    "appeals_timely", "appeals_review", "ttyt_available"
]

# Column names for star rating domain files (consistent 2010-2015)
STAR_COLS = (
    ["contractid", "org_type", "contract_name", "org_marketing"] +
    RATING_VARS
)

print(f"Using {len(RATING_VARS)} rating measures.")